# Fees analysis computation

#### This following function computes and creates analysis files automatically, from the csv download on Stripe.

In [1]:
# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import date
import sklearn
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from pathlib import Path  

In [3]:
def stripe(file_global, radar) :
    df = pd.read_csv(file_global)
    df_bis = pd.read_csv(radar)

    # Global file analysis

    df.card_brand = df['card_brand'].fillna(value='unknown')
    df.card_funding = df['card_funding'].fillna(value='unknown')
    df.plan_name = df['plan_name'].fillna(value='unknown')

    to_dl = df[['card_brand','card_funding','fee_category','fee_name','event_type','plan_name','variable_rate','fixed_per_item_amount','variable_volume_amount','total_amount','tax_amount','balance_transaction_id']].groupby(['event_type','card_brand','card_funding','fee_category','fee_name','plan_name']
            ).agg(
        TOTAL_fixed_fees=('fixed_per_item_amount', 'sum'),
        TOTAL_volume_variable_fees=('variable_volume_amount', 'sum'),
        TOTAL_fees_amount=('total_amount', 'sum'),
        MEAN_fixed_fees=('fixed_per_item_amount', 'mean'),
        MEAN_variable_rate=('variable_rate', 'mean'),
        by_transaction_occuring=('balance_transaction_id','nunique'),
        total_occuring = ('balance_transaction_id','count')
    )

    to_dl.to_csv('stripe_p&l.csv')

    # Radar, 3DS and payout file analysis

    df_bis.card_brand = df_bis['card_brand'].fillna(value='unknown')
    df_bis.card_funding = df_bis['card_brand'].fillna(value='unknown')
    df_bis['month_year'] = pd.to_datetime(df_bis['activity_at']).dt.strftime('%m-%Y')

    A = df_bis[df_bis['balance_transaction_description'].str.contains('Payout', case=False)]
    B = df_bis[df_bis['balance_transaction_description'].str.contains('3D', case=False)]
    C = df_bis[df_bis['balance_transaction_description'].str.contains('Radar', case=False)]

    tot = pd.concat([A,B,C], axis=0)

    to_dl = tot[['balance_transaction_description','amount']].groupby('balance_transaction_description')[['amount']].sum()

    to_dl.to_csv('stripe_p&l_RADAR.csv')


In [4]:
stripe('Oct_fees_sumary.csv','Oct_stripe_RADAR.csv')

/var/folders/kx/053l91k97ggc1ywh6t0pvckh0000gn/T/ipykernel_5824/2078803205.py:2: DtypeWarning: Columns (6,16,19,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_global)
